<a href="https://colab.research.google.com/github/AnkitaChakraborty1/Capstone_Adversarial_Robustness/blob/main/Custom_Recipe_list_Allen_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##pip installs

In [2]:
pip install transformers

In [3]:
!pip3 install textattack[tensorflow]

  Using cached numpy-1.19.5-cp37-cp37m-manylinux2010_x86_64.whl (14.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [1]:
pip install --upgrade numpy

  Using cached numpy-1.21.5-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.21.5 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [1]:
pip install folium==0.2.1

In [2]:
!pip install allennlp allennlp_models > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
textattack 0.3.4 requires tqdm<4.50.0,>=4.27, but you have tqdm 4.62.3 which is incompatible.


In [3]:
pip install --upgrade google-cloud-storage

     |████████████████████████████████| 106 kB 7.1 MB/s 
     |████████████████████████████████| 111 kB 42.8 MB/s 
     |████████████████████████████████| 75 kB 5.1 MB/s 
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.26.3
    Uninstalling google-api-core-1.26.3:
      Successfully uninstalled google-api-core-1.26.3
  Attempting uninstall: google-resumable-media
    Found existing installation: google-resumable-media 0.4.1
    Uninstalling google-resumable-media-0.4.1:
      Successfully uninstalled google-resumable-media-0.4.1
  Attempting uninstall: google-cloud-core
    Found existing installation: google-cloud-core 1.0.3
    Uninstalling google-cloud-core-1.0.3:
      Successfully uninstalled google-cloud-core-1.0.3
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 1.18.1
    Uninstalling google-cloud-storage-1.18.1:
      Successfully uninstalled google-cloud-storage-1.18.1
ERROR: pip's de

## Creating the AllenNLP Model

In [4]:
from allennlp.predictors import Predictor
import allennlp_models.classification

import textattack

class AllenNLPModel(textattack.models.wrappers.ModelWrapper):
    def __init__(self):
        self.predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/basic_stanford_sentiment_treebank-2020.06.09.tar.gz")
        self.model = self.predictor._model
        self.tokenizer = self.predictor._dataset_reader._tokenizer

    def __call__(self, text_input_list):
        outputs = []
        for text_input in text_input_list:
            outputs.append(self.predictor.predict(sentence=text_input))
        # For each output, outputs['logits'] contains the logits where
        # index 0 corresponds to the positive and index 1 corresponds 
        # to the negative score. We reverse the outputs (by reverse slicing,
        # [::-1]) so that negative comes first and positive comes second.
        return [output['logits'][::-1] for output in outputs]

model_wrapper = AllenNLPModel()

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:14<00:00, 34.0MB/s]
textattack: Unzipping file /root/.cache/textattack

In [5]:
pip install torchfile

  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5709 sha256=754057b85c8a28aa3c080b45d910a1a61ac7b1488c227ac1a4d6a133c4d85627
  Stored in directory: /root/.cache/pip/wheels/ac/5c/3a/a80e1c65880945c71fd833408cd1e9a8cb7e2f8f37620bb75b
Successfully built torchfile


In [53]:
from textattack import AttackArgs, Attacker
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import ModelWrapper
from textattack import Attack
from textattack.constraints.pre_transformation import (
        InputColumnModification,
        MaxModificationRate,
        RepeatModification,
        StopwordModification,
        MaxWordIndexModification
                )
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR,AlzantotGeneticAlgorithm
from textattack.transformations import WordSwapQWERTY
from textattack.transformations import (
    CompositeTransformation,
    WordSwapNeighboringCharacterSwap,
    WordSwapQWERTY,
    WordSwapWordNet,
    WordSwapRandomCharacterDeletion,
    WordSwapRandomCharacterInsertion,
    WordSwapRandomCharacterSubstitution,
    WordSwapHomoglyphSwap,
    WordSwapEmbedding
)
from textattack.constraints.overlap import LevenshteinEditDistance
#from textattack.attack_recipe import AttackRecipe

from abc import ABC, abstractmethod

In [56]:
from textattack.datasets import HuggingFaceDataset
from textattack.attacker import Attacker
from textattack import AttackArgs, Attacker
from textattack.datasets import HuggingFaceDataset
from textattack.goal_functions import TargetedClassification

import torch


torch.multiprocessing.freeze_support()
dataset = HuggingFaceDataset("glue", "sst2", "train")  

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

textattack: Loading datasets dataset glue, subset sst2, split train.


##Custom Recipes

##Recipe1    
            WordSwapQWERTY
            WordSwapWordNet
            WordSwapHomoglyphSwap
            WordSwapNeighboringCharacterSwap
            WordSwapRandomCharacterInsertion
            WordSwapRandomCharacterDeletion
            WordSwapEmbedding
            WordEmbedding
      

In [9]:
class Recipe1(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
        #transformation =  WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        transformation = CompositeTransformation(
                [
                    WordSwapQWERTY(),
                    WordSwapWordNet(),
                    WordSwapHomoglyphSwap(),
                    WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,),
                    # (1) Insert: Insert a space into the word.
                    # Generally, words are segmented by spaces in English. Therefore,
                    # we can deceive classifiers by inserting spaces into words.
                    WordSwapRandomCharacterInsertion(
                        random_one=True,
                        letters_to_insert=" ",
                        skip_first_char=True,
                        skip_last_char=True,
                    ),
                    # (2) Delete: Delete a random character of the word except for the first
                    # and the last character
                    WordSwapRandomCharacterDeletion(
                        random_one=True, skip_first_char=True, skip_last_char=True
                    ),
                    # (5) Substitute-W
                    # (Sub-W): Replace a word with its topk nearest neighbors in a
                    # context-aware word vector space. Specifically, we use the pre-trained
                    # GloVe model [30] provided by Stanford for word embedding and set
                    # topk = 5 in the experiment.
                    WordSwapEmbedding(max_candidates=5),
                ])

        constraints = [RepeatModification(), StopwordModification()]

        goal_function = UntargetedClassification(model_wrapper)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)


In [7]:
attack = Recipe1.build(model_wrapper)
attacker = Attacker(attack, dataset)
attacker.attack_dataset()

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.44M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

...

--------------------------------------------- Result 6 ---------------------------------------------
[[Negative (99%)]] --> [[Positive (72%)]]

that 's far too [[tragic]] to merit such [[superficial]] treatment 

that 's far too [[tr]] [[agic]] to merit such [[cosmetic]] treatment 




[Succeeded / Failed / Skipped / Total] 7 / 0 / 1 / 8:  80%|████████  | 8/10 [00:03<00:00,  2.29it/s]

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (98%)]] --> [[Negative (81%)]]

[[demonstrates]] that the director of such [[hollywood]] blockbusters as patriot games can still turn out a small , personal [[film]] with an emotional [[wallop]] . 

[[evidence]] that the director of such [[tinseltown]] blockbusters as patriot games can still turn out a small , personal [[shoot]] with an emotional [[batting]] . 


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (90%)]] --> [[Negative (81%)]]

of [[saucy]] 

of [[impudent]] 




[Succeeded / Failed / Skipped / Total] 9 / 0 / 1 / 10: 100%|██████████| 10/10 [00:03<00:00,  2.52it/s]

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (99%)]] --> [[Positive (74%)]]

a [[depressed]] fifteen-year-old 's [[suicidal]] poetry 

a [[low-spirited]] fifteen-year-old 's [[suici]] [[dal]] poetry 


--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (79%)]] --> [[Negative (97%)]]

are more [[deeply]] thought through than in most ` right-thinking ' films 

are more [[severely]] thought through than in most ` right-thinking ' films 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 9      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 42.86% |


##Recipe2
          WordSwapQWERTY
          WordSwapWordNet
          WordSwapHomoglyphSwap
          WordSwapNeighboringCharacterSwap
          WordSwapRandomCharacterInsertion
          WordSwapRandomCharacterDeletion       

In [11]:
class Recipe2(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
        #transformation =  WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        transformation = CompositeTransformation(
                [
                    WordSwapQWERTY(),
                    WordSwapWordNet(),
                    WordSwapHomoglyphSwap(),
                    WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,),
                    # (1) Insert: Insert a space into the word.
                    # Generally, words are segmented by spaces in English. Therefore,
                    # we can deceive classifiers by inserting spaces into words.
                    WordSwapRandomCharacterInsertion(
                        random_one=True,
                        letters_to_insert=" ",
                        skip_first_char=True,
                        skip_last_char=True,
                    ),
                    # (2) Delete: Delete a random character of the word except for the first
                    # and the last character
                    WordSwapRandomCharacterDeletion(
                        random_one=True, skip_first_char=True, skip_last_char=True
                    ),
                ])

        constraints = [RepeatModification(), StopwordModification()]

        goal_function = UntargetedClassification(model_wrapper)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

In [12]:
attack = Recipe2.build(model_wrapper)
attacker = Attacker(attack, dataset)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'allennlp.models.basic_classifier.BasicClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapQWERTY
    (1): WordSwapWordNet
    (2): WordSwapHomoglyphSwap
    (3): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (4): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (5): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:00<00:01,  7.74it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (95%)]] --> [[Positive (93%)]]

[[hide]] new secretions from the parental units 

[[enshroud]] new secretions from the parental units 




[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:  20%|██        | 2/10 [00:00<00:02,  3.46it/s]

...

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (98%)]] --> [[Negative (54%)]]

[[demonstrates]] that the [[director]] of such [[hollywood]] blockbusters as [[patriot]] games can still turn out a [[small]] , personal [[film]] with an emotional wallop . 

[[evidence]] that the [[manager]] of such [[Hollywood]] blockbusters as [[pat]] [[riot]] games can still turn out a [[lowly]] , personal [[shoot]] with an emotional wallop . 


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (90%)]] --> [[Negative (81%)]]

of [[saucy]] 

of [[impudent]] 




[Succeeded / Failed / Skipped / Total] 8 / 0 / 1 / 9:  90%|█████████ | 9/10 [00:02<00:00,  3.10it/s]

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (99%)]] --> [[Positive (74%)]]

a [[depressed]] fifteen-year-old 's [[suicidal]] poetry 

a [[low-spirited]] fifteen-year-old 's [[suici]] [[dal]] poetry 




[Succeeded / Failed / Skipped / Total] 9 / 0 / 1 / 10: 100%|██████████| 10/10 [00:03<00:00,  3.18it/s]

--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (79%)]] --> [[Negative (51%)]]

are more [[deeply]] thought through than in most ` right-thinking ' [[films]] 

are more [[profoundly]] thought through than in most ` right-thinking ' [[celluloid]] 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 9      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 47.93% |
| Average num. words per input: | 9.5    |
| Avg num queries:              | 61.89  |
+-------------------------------+--------+


##Recipe3
          WordSwapWordNet
          WordSwapHomoglyphSwap
          WordSwapNeighboringCharacterSwap
          WordSwapRandomCharacterInsertion
          WordSwapRandomCharacterDeletion
          WordSwapEmbedding

In [13]:
class Recipe3(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
        #transformation =  WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        transformation = CompositeTransformation(
                [
                    WordSwapWordNet(),
                    WordSwapHomoglyphSwap(),
                    WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,),
                    # (1) Insert: Insert a space into the word.
                    # Generally, words are segmented by spaces in English. Therefore,
                    # we can deceive classifiers by inserting spaces into words.
                    WordSwapRandomCharacterInsertion(
                        random_one=True,
                        letters_to_insert=" ",
                        skip_first_char=True,
                        skip_last_char=True,
                    ),
                    # (2) Delete: Delete a random character of the word except for the first
                    # and the last character
                    WordSwapRandomCharacterDeletion(
                        random_one=True, skip_first_char=True, skip_last_char=True
                    ),
                    # (5) Substitute-W
                    # (Sub-W): Replace a word with its topk nearest neighbors in a
                    # context-aware word vector space. Specifically, we use the pre-trained
                    # GloVe model [30] provided by Stanford for word embedding and set
                    # topk = 5 in the experiment.
                    WordSwapEmbedding(max_candidates=5),
                ])

        constraints = [RepeatModification(), StopwordModification()]

        goal_function = UntargetedClassification(model_wrapper)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)


In [14]:
attack = Recipe3.build(model_wrapper)
attacker = Attacker(attack, dataset)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'allennlp.models.basic_classifier.BasicClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapWordNet
    (1): WordSwapHomoglyphSwap
    (2): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (3): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (4): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (5): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:00<00:01,  6.54it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (95%)]] --> [[Positive (93%)]]

[[hide]] new secretions from the parental units 

[[concealing]] new secretions from the parental units 




[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:  20%|██        | 2/10 [00:00<00:02,  2.95it/s]

...

--------------------------------------------- Result 6 ---------------------------------------------
[[Negative (99%)]] --> [[Positive (71%)]]

that 's [[far]] too [[tragic]] to merit such [[superficial]] [[treatment]] 

that 's [[ALIR]] too [[calamitous]] to merit such [[cosmetic]] [[tr]] [[eatment]] 




[Succeeded / Failed / Skipped / Total] 7 / 0 / 1 / 8:  80%|████████  | 8/10 [00:02<00:00,  2.81it/s]

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (98%)]] --> [[Negative (56%)]]

[[demonstrates]] that the director of such [[hollywood]] blockbusters as [[patriot]] games can still turn out a small , [[personal]] [[film]] with an [[emotional]] wallop . 

[[evidence]] that the director of such [[tinseltown]] blockbusters as [[pat]] [[riot]] games can still turn out a small , [[individual]] [[shoot]] with an [[sentimental]] wallop . 


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (90%)]] --> [[Negative (81%)]]

of [[saucy]] 

of [[impudent]] 




[Succeeded / Failed / Skipped / Total] 9 / 0 / 1 / 10: 100%|██████████| 10/10 [00:03<00:00,  3.01it/s]

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (99%)]] --> [[Positive (74%)]]

a [[depressed]] fifteen-year-old 's [[suicidal]] poetry 

a [[low-spirited]] fifteen-year-old 's [[su]] [[icidal]] poetry 


--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (79%)]] --> [[Negative (97%)]]

are more [[deeply]] thought through than in most ` right-thinking ' films 

are more [[severely]] thought through than in most ` right-thinking ' films 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 9      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 46.44% |


##Recipe 4

In [15]:
class Recipe4(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
        #transformation =  WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        transformation = CompositeTransformation(
                [
                    WordSwapWordNet(),

                    WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,),
                    # (1) Insert: Insert a space into the word.
                    # Generally, words are segmented by spaces in English. Therefore,
                    # we can deceive classifiers by inserting spaces into words.
                    WordSwapRandomCharacterInsertion(
                        random_one=True,
                        letters_to_insert=" ",
                        skip_first_char=True,
                        skip_last_char=True,
                    ),
                    # (2) Delete: Delete a random character of the word except for the first
                    # and the last character
                    WordSwapRandomCharacterDeletion(
                        random_one=True, skip_first_char=True, skip_last_char=True
                    ),
                    # (5) Substitute-W
                    # (Sub-W): Replace a word with its topk nearest neighbors in a
                    # context-aware word vector space. Specifically, we use the pre-trained
                    # GloVe model [30] provided by Stanford for word embedding and set
                    # topk = 5 in the experiment.
                    WordSwapEmbedding(max_candidates=5),
                ])

        constraints = [RepeatModification(), StopwordModification()]

        goal_function = UntargetedClassification(model_wrapper)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

In [16]:
attack = Recipe4.build(model_wrapper)
attacker = Attacker(attack, dataset)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'allennlp.models.basic_classifier.BasicClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapWordNet
    (1): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (3): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (4): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:00<00:01,  7.58it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (95%)]] --> [[Positive (93%)]]

[[hide]] new secretions from the parental units 

[[concealing]] new secretions from the parental units 




[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:  20%|██        | 2/10 [00:00<00:02,  3.53it/s]

...

--------------------------------------------- Result 6 ---------------------------------------------
[[Negative (99%)]] --> [[Positive (71%)]]

that 's [[far]] too [[tragic]] to merit such [[superficial]] [[treatment]] 

that 's [[ALIR]] too [[calamitous]] to merit such [[cosmetic]] [[tr]] [[eatment]] 




[Succeeded / Failed / Skipped / Total] 7 / 0 / 1 / 8:  80%|████████  | 8/10 [00:02<00:00,  3.89it/s]

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (98%)]] --> [[Negative (56%)]]

[[demonstrates]] that the director of such [[hollywood]] blockbusters as [[patriot]] games can still turn out a small , [[personal]] [[film]] with an [[emotional]] wallop . 

[[evidence]] that the director of such [[tinseltown]] blockbusters as [[pat]] [[riot]] games can still turn out a small , [[individual]] [[shoot]] with an [[sentimental]] wallop . 


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (90%)]] --> [[Negative (81%)]]

of [[saucy]] 

of [[impudent]] 




[Succeeded / Failed / Skipped / Total] 9 / 0 / 1 / 10: 100%|██████████| 10/10 [00:02<00:00,  4.18it/s]

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (99%)]] --> [[Positive (74%)]]

a [[depressed]] fifteen-year-old 's [[suicidal]] poetry 

a [[low-spirited]] fifteen-year-old 's [[su]] [[icidal]] poetry 


--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (79%)]] --> [[Negative (97%)]]

are more [[deeply]] thought through than in most ` right-thinking ' films 

are more [[severely]] thought through than in most ` right-thinking ' films 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 9      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 46.44% |


##Recipe5
          WordSwapQWERTY
          WordSwapWordNet
          WordSwapHomoglyphSwap
          WordSwapNeighboringCharacterSwap
          WordSwapRandomCharacterInsertion

In [17]:
class Recipe5(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
        #transformation =  WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        transformation = CompositeTransformation(
                [
                    WordSwapQWERTY(),
                    WordSwapWordNet(),
                    WordSwapHomoglyphSwap(),
                    WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,),
                    # (1) Insert: Insert a space into the word.
                    # Generally, words are segmented by spaces in English. Therefore,
                    # we can deceive classifiers by inserting spaces into words.
                    WordSwapRandomCharacterInsertion(
                        random_one=True,
                        letters_to_insert=" ",
                        skip_first_char=True,
                        skip_last_char=True,
                    ),
                ])

        constraints = [RepeatModification(), StopwordModification()]

        goal_function = UntargetedClassification(model_wrapper)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)


In [18]:
attack = Recipe5.build(model_wrapper)
attacker = Attacker(attack, dataset)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'allennlp.models.basic_classifier.BasicClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapQWERTY
    (1): WordSwapWordNet
    (2): WordSwapHomoglyphSwap
    (3): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (4): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:00<00:01,  7.54it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (95%)]] --> [[Positive (93%)]]

[[hide]] new secretions from the parental units 

[[enshroud]] new secretions from the parental units 




[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:  20%|██        | 2/10 [00:00<00:02,  3.49it/s]

...

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (98%)]] --> [[Negative (58%)]]

[[demonstrates]] that the [[director]] of such [[hollywood]] blockbusters as patriot games can still turn out a small , personal [[film]] with an emotional [[wallop]] . 

[[evidence]] that the [[manager]] of such [[Hollywood]] blockbusters as patriot games can still turn out a small , personal [[shoot]] with an emotional [[w]] [[allop]] . 


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (90%)]] --> [[Negative (81%)]]

of [[saucy]] 

of [[impudent]] 




[Succeeded / Failed / Skipped / Total] 7 / 1 / 1 / 9:  90%|█████████ | 9/10 [00:02<00:00,  3.67it/s]

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (99%)]] --> [[Positive (74%)]]

a [[depressed]] fifteen-year-old 's [[suicidal]] poetry 

a [[low-spirited]] fifteen-year-old 's [[su]] [[icidal]] poetry 




[Succeeded / Failed / Skipped / Total] 8 / 1 / 1 / 10: 100%|██████████| 10/10 [00:02<00:00,  3.74it/s]

--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (79%)]] --> [[Negative (51%)]]

are more [[deeply]] thought through than in most ` right-thinking ' [[films]] 

are more [[profoundly]] thought through than in most ` right-thinking ' [[celluloid]] 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 1      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 10.0%  |
| Attack success rate:          | 88.89% |
| Average perturbed word %:     | 37.4%  |
| Average num. words per input: | 9.5    |
| Avg num queries:              | 52.67  |
+-------------------------------+--------+


##Recipe 6
          WordSwapNeighboringCharacterSwap
          WordSwapRandomCharacterInsertion
          WordSwapRandomCharacterDeletion
          WordSwapEmbedding
          WordEmbedding

In [19]:
class Recipe6(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
        #transformation =  WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        transformation = CompositeTransformation(
                [
                    WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,),
                    # (1) Insert: Insert a space into the word.
                    # Generally, words are segmented by spaces in English. Therefore,
                    # we can deceive classifiers by inserting spaces into words.
                    WordSwapRandomCharacterInsertion(
                        random_one=True,
                        letters_to_insert=" ",
                        skip_first_char=True,
                        skip_last_char=True,
                    ),
                    # (2) Delete: Delete a random character of the word except for the first
                    # and the last character
                    WordSwapRandomCharacterDeletion(
                        random_one=True, skip_first_char=True, skip_last_char=True
                    ),
                    # (5) Substitute-W
                    # (Sub-W): Replace a word with its topk nearest neighbors in a
                    # context-aware word vector space. Specifically, we use the pre-trained
                    # GloVe model [30] provided by Stanford for word embedding and set
                    # topk = 5 in the experiment.
                    WordSwapEmbedding(max_candidates=5),
                ])

        constraints = [RepeatModification(), StopwordModification()]

        goal_function = UntargetedClassification(model_wrapper)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

In [20]:
attack = Recipe6.build(model_wrapper)
attacker = Attacker(attack, dataset)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'allennlp.models.basic_classifier.BasicClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (3): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:00<00:00, 11.88it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (95%)]] --> [[Positive (93%)]]

[[hide]] new secretions from the parental units 

[[concealing]] new secretions from the parental units 




[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:  20%|██        | 2/10 [00:00<00:01,  7.16it/s]

...

--------------------------------------------- Result 6 ---------------------------------------------
[[Negative (99%)]] --> [[Positive (72%)]]

that 's far too [[tragic]] to merit such [[superficial]] treatment 

that 's far too [[trag]] [[ic]] to merit such [[cosmetic]] treatment 




[Succeeded / Failed / Skipped / Total] 7 / 1 / 1 / 9:  90%|█████████ | 9/10 [00:01<00:00,  5.43it/s]

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (98%)]] --> [[Negative (72%)]]

[[demonstrates]] that the [[director]] of such [[hollywood]] blockbusters as [[patriot]] games can still turn out a [[small]] , [[personal]] [[film]] with an [[emotional]] wallop . 

[[shows]] that the [[directors]] of such [[tinseltown]] blockbusters as [[pat]] [[riot]] games can still turn out a [[scant]] , [[individual]] [[movies]] with an [[sentimental]] wallop . 


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (90%)]] --> [[Negative (53%)]]

of [[saucy]] 

of [[smutty]] 


--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (99%)]] --> [[Positive (69%)]]

a [[depressed]] fifteen-year-old 's [[suicidal]] poetry 

a [[dep]] [[ressed]] fifteen-year-old 's [[suici]] [[dal]] poetry 




[Succeeded / Failed / Skipped / Total] 8 / 1 / 1 / 10: 100%|██████████| 10/10 [00:01<00:00,  5.64it/s]

--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (79%)]] --> [[Negative (97%)]]

are more [[deeply]] thought through than in most ` right-thinking ' films 

are more [[severely]] thought through than in most ` right-thinking ' films 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 1      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 10.0%  |
| Attack success rate:          | 88.89% |
| Average perturbed word %:     | 49.04% |
| Average num. words per input: | 9.5    |
| Avg num queries:              | 33.56  |
+-------------------------------+--------+


##Recipe 7
          WordSwapQWERTY
          WordSwapWordNet
          WordSwapHomoglyphSwap
          WordSwapNeighboringCharacterSwap

In [21]:
class Recipe7(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
        #transformation =  WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        transformation = CompositeTransformation(
                [
                    WordSwapQWERTY(),
                    WordSwapWordNet(),
                    WordSwapHomoglyphSwap(),
                    WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,),
                ])

        constraints = [RepeatModification(), StopwordModification()]

        goal_function = UntargetedClassification(model_wrapper)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

In [22]:
attack = Recipe7.build(model_wrapper)
attacker = Attacker(attack, dataset)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'allennlp.models.basic_classifier.BasicClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapQWERTY
    (1): WordSwapWordNet
    (2): WordSwapHomoglyphSwap
    (3): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:00<00:01,  8.32it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (95%)]] --> [[Positive (93%)]]

[[hide]] new secretions from the parental units 

[[enshroud]] new secretions from the parental units 




[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:  20%|██        | 2/10 [00:00<00:02,  3.63it/s]

...

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (98%)]] --> [[Negative (50%)]]

[[demonstrates]] that the [[director]] of such [[hollywood]] blockbusters as patriot games can still turn out a small , personal [[film]] with an emotional [[wallop]] . 

[[evidence]] that the [[manager]] of such [[Hollywood]] blockbusters as patriot games can still turn out a small , personal [[shoot]] with an emotional [[impact]] . 


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (90%)]] --> [[Negative (81%)]]

of [[saucy]] 

of [[impudent]] 




[Succeeded / Failed / Skipped / Total] 8 / 1 / 1 / 10: 100%|██████████| 10/10 [00:02<00:00,  3.67it/s]

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (99%)]] --> [[Positive (90%)]]

a [[depressed]] [[fifteen-year-old]] 's [[suicidal]] poetry 

a [[low-spirited]] [[fifteen˗year-old]] 's [[suicidaⅼ]] poetry 


--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (79%)]] --> [[Negative (51%)]]

are more [[deeply]] thought through than in most ` right-thinking ' [[films]] 

are more [[profoundly]] thought through than in most ` right-thinking ' [[celluloid]] 




[Succeeded / Failed / Skipped / Total] 8 / 1 / 1 / 10: 100%|██████████| 10/10 [00:02<00:00,  3.67it/s]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 1      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 10.0%  |
| Attack success rate:          | 88.89% |
| Average perturbed word %:     | 29.9%  |
| Average num. words per input: | 9.5    |
| Avg num queries:              | 54.33  |
+-------------------------------+--------+


##Recipe 8
            WordSwapRandomCharacterInsertion
            WordSwapRandomCharacterDeletion
            WordSwapEmbedding

In [24]:
class Recipe8(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
        #transformation =  WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        transformation = CompositeTransformation(
                [
                    # (1) Insert: Insert a space into the word.
                    # Generally, words are segmented by spaces in English. Therefore,
                    # we can deceive classifiers by inserting spaces into words.
                    WordSwapRandomCharacterInsertion(
                        random_one=True,
                        letters_to_insert=" ",
                        skip_first_char=True,
                        skip_last_char=True,
                    ),
                    # (2) Delete: Delete a random character of the word except for the first
                    # and the last character
                    WordSwapRandomCharacterDeletion(
                        random_one=True, skip_first_char=True, skip_last_char=True
                    ),
                    # (5) Substitute-W
                    # (Sub-W): Replace a word with its topk nearest neighbors in a
                    # context-aware word vector space. Specifically, we use the pre-trained
                    # GloVe model [30] provided by Stanford for word embedding and set
                    # topk = 5 in the experiment.
                    WordSwapEmbedding(max_candidates=5),
                ])

        constraints = [RepeatModification(), StopwordModification()]

        goal_function = UntargetedClassification(model_wrapper)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

In [25]:
attack = Recipe8.build(model_wrapper)
attacker = Attacker(attack, dataset)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'allennlp.models.basic_classifier.BasicClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:00<00:00, 12.50it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (95%)]] --> [[Positive (93%)]]

[[hide]] new secretions from the parental units 

[[concealing]] new secretions from the parental units 




[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:  20%|██        | 2/10 [00:00<00:01,  7.68it/s]

...

--------------------------------------------- Result 6 ---------------------------------------------
[[Negative (99%)]] --> [[Positive (71%)]]

that 's [[far]] too [[tragic]] to merit such [[superficial]] [[treatment]] 

that 's [[fr]] too [[calamitous]] to merit such [[cosmetic]] [[treatme]] [[nt]] 




[Succeeded / Failed / Skipped / Total] 7 / 1 / 1 / 9:  90%|█████████ | 9/10 [00:01<00:00,  5.93it/s]

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (98%)]] --> [[Negative (54%)]]

[[demonstrates]] that the [[director]] of such [[hollywood]] blockbusters as patriot games can still turn out a small , personal [[film]] with an emotional [[wallop]] . 

[[shows]] that the [[dire]] [[ctor]] of such [[tinseltown]] blockbusters as patriot games can still turn out a small , personal [[movies]] with an emotional [[batting]] . 


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (90%)]] --> [[Negative (53%)]]

of [[saucy]] 

of [[smutty]] 


--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (99%)]] --> [[Positive (69%)]]

a [[depressed]] fifteen-year-old 's [[suicidal]] poetry 

a [[depre]] [[ssed]] fifteen-year-old 's [[suici]] [[dal]] poetry 




[Succeeded / Failed / Skipped / Total] 8 / 1 / 1 / 10: 100%|██████████| 10/10 [00:01<00:00,  6.12it/s]

--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (79%)]] --> [[Negative (97%)]]

are more [[deeply]] thought through than in most ` right-thinking ' films 

are more [[severely]] thought through than in most ` right-thinking ' films 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 1      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 10.0%  |
| Attack success rate:          | 88.89% |
| Average perturbed word %:     | 48.71% |
| Average num. words per input: | 9.5    |
| Avg num queries:              | 30.89  |
+-------------------------------+--------+


##Recipe 9
          WordSwapRandomCharacterInsertion
          WordSwapRandomCharacterDeletion
          WordSwapEmbedding

In [45]:
class Recipe9(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
        #transformation =  WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        transformation = CompositeTransformation(
                [
                    # (1) Insert: Insert a space into the word.
                    # Generally, words are segmented by spaces in English. Therefore,
                    # we can deceive classifiers by inserting spaces into words.
                    WordSwapRandomCharacterInsertion(
                        random_one=True,
                        letters_to_insert=" ",
                        skip_first_char=True,
                        skip_last_char=True,
                    ),
                    # (2) Delete: Delete a random character of the word except for the first
                    # and the last character
                    WordSwapRandomCharacterDeletion(
                        random_one=True, skip_first_char=True, skip_last_char=True
                    ),
                    # (5) Substitute-W
                    # (Sub-W): Replace a word with its topk nearest neighbors in a
                    # context-aware word vector space. Specifically, we use the pre-trained
                    # GloVe model [30] provided by Stanford for word embedding and set
                    # topk = 5 in the experiment.
                    WordSwapEmbedding(max_candidates=5),
                ])

        constraints = [RepeatModification(), StopwordModification()]

        goal_function = UntargetedClassification(model_wrapper)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

In [46]:
attack = Recipe9.build(model_wrapper)
attacker = Attacker(attack, dataset)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'allennlp.models.basic_classifier.BasicClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:00<00:00, 11.92it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (95%)]] --> [[Positive (93%)]]

[[hide]] new secretions from the parental units 

[[concealing]] new secretions from the parental units 




[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:  20%|██        | 2/10 [00:00<00:01,  7.54it/s]

...

--------------------------------------------- Result 6 ---------------------------------------------
[[Negative (99%)]] --> [[Positive (71%)]]

that 's [[far]] too [[tragic]] to merit such [[superficial]] [[treatment]] 

that 's [[fr]] too [[calamitous]] to merit such [[cosmetic]] [[treatme]] [[nt]] 




[Succeeded / Failed / Skipped / Total] 7 / 1 / 1 / 9:  90%|█████████ | 9/10 [00:01<00:00,  6.06it/s]

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (98%)]] --> [[Negative (54%)]]

[[demonstrates]] that the [[director]] of such [[hollywood]] blockbusters as patriot games can still turn out a small , personal [[film]] with an emotional [[wallop]] . 

[[shows]] that the [[dire]] [[ctor]] of such [[tinseltown]] blockbusters as patriot games can still turn out a small , personal [[movies]] with an emotional [[batting]] . 


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (90%)]] --> [[Negative (53%)]]

of [[saucy]] 

of [[smutty]] 


--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (99%)]] --> [[Positive (69%)]]

a [[depressed]] fifteen-year-old 's [[suicidal]] poetry 

a [[depre]] [[ssed]] fifteen-year-old 's [[suici]] [[dal]] poetry 




[Succeeded / Failed / Skipped / Total] 8 / 1 / 1 / 10: 100%|██████████| 10/10 [00:01<00:00,  6.27it/s]

--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (79%)]] --> [[Negative (97%)]]

are more [[deeply]] thought through than in most ` right-thinking ' films 

are more [[severely]] thought through than in most ` right-thinking ' films 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 1      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 10.0%  |
| Attack success rate:          | 88.89% |
| Average perturbed word %:     | 48.71% |
| Average num. words per input: | 9.5    |
| Avg num queries:              | 30.89  |
+-------------------------------+--------+


##Recipe 10
            WordSwapQWERTY
            WordSwapWordNet
            WordSwapNeighboringCharacterSwap

In [32]:
class Recipe10(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
        #transformation =  WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        transformation = CompositeTransformation(
                [
                    WordSwapQWERTY(),
                    WordSwapWordNet(),
                    WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
                ])

        constraints = [RepeatModification(), StopwordModification()]

        goal_function = UntargetedClassification(model_wrapper)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

In [33]:
attack = Recipe10.build(model_wrapper)
attacker = Attacker(attack, dataset)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'allennlp.models.basic_classifier.BasicClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapQWERTY
    (1): WordSwapWordNet
    (2): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:00<00:00,  9.62it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (95%)]] --> [[Positive (93%)]]

[[hide]] new secretions from the parental units 

[[enshroud]] new secretions from the parental units 




[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:  20%|██        | 2/10 [00:00<00:01,  4.80it/s]

...

--------------------------------------------- Result 6 ---------------------------------------------
[[Negative (99%)]] --> [[Positive (64%)]]

that 's [[far]] too [[tragic]] to merit such [[superficial]] [[treatment]] 

that 's [[ALIR]] too [[targic]] to merit such [[superficail]] [[treamtent]] 




[Succeeded / Failed / Skipped / Total] 6 / 1 / 1 / 8:  80%|████████  | 8/10 [00:01<00:00,  5.84it/s]

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (98%)]] --> [[Negative (50%)]]

[[demonstrates]] that the [[director]] of such [[hollywood]] blockbusters as patriot games can still turn out a small , personal [[film]] with an emotional [[wallop]] . 

[[evidence]] that the [[manager]] of such [[Hollywood]] blockbusters as patriot games can still turn out a small , personal [[shoot]] with an emotional [[impact]] . 


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (90%)]] --> [[Negative (81%)]]

of [[saucy]] 

of [[impudent]] 




[Succeeded / Failed / Skipped / Total] 8 / 1 / 1 / 10: 100%|██████████| 10/10 [00:01<00:00,  5.99it/s]

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (99%)]] --> [[Positive (84%)]]

a [[depressed]] [[fifteen-year-old]] 's [[suicidal]] poetry 

a [[low-spirited]] [[fiftene-year-old]] 's [[suicldal]] poetry 


--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (79%)]] --> [[Negative (51%)]]

are more [[deeply]] thought through than in most ` right-thinking ' [[films]] 

are more [[profoundly]] thought through than in most ` right-thinking ' [[celluloid]] 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 1      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 10.0%  |
| Attack success rate:          | 88.89% |
| Average perturbed word %

##Recipe 11
            WordSwapRandomCharacterDeletion
            WordSwapEmbedding

In [49]:
class Recipe11(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
        #transformation =  WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        transformation = CompositeTransformation(
                [
                    # (2) Delete: Delete a random character of the word except for the first
                    # and the last character
                    WordSwapRandomCharacterDeletion(
                        random_one=True, skip_first_char=True, skip_last_char=True
                    ),
                    # (5) Substitute-W
                    # (Sub-W): Replace a word with its topk nearest neighbors in a
                    # context-aware word vector space. Specifically, we use the pre-trained
                    # GloVe model [30] provided by Stanford for word embedding and set
                    # topk = 5 in the experiment.
                    WordSwapEmbedding(max_candidates=5),
                ])

        constraints = [RepeatModification(), StopwordModification()]

        goal_function = UntargetedClassification(model_wrapper)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)


In [50]:
attack = Recipe11.build(model_wrapper)
attacker = Attacker(attack, dataset)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'allennlp.models.basic_classifier.BasicClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (1): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:00<00:00, 13.96it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (95%)]] --> [[Positive (93%)]]

[[hide]] new secretions from the parental units 

[[concealing]] new secretions from the parental units 




[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:  20%|██        | 2/10 [00:00<00:00,  9.14it/s]

...

--------------------------------------------- Result 6 ---------------------------------------------
[[Negative (99%)]] --> [[Positive (64%)]]

that 's [[far]] too [[tragic]] to merit such [[superficial]] [[treatment]] 

that 's [[fr]] too [[calamitous]] to merit such [[cosmetic]] [[processing]] 




[Succeeded / Failed / Skipped / Total] 7 / 1 / 1 / 9:  90%|█████████ | 9/10 [00:01<00:00,  6.53it/s]

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (98%)]] --> [[Negative (54%)]]

[[demonstrates]] that the [[director]] of such [[hollywood]] blockbusters as patriot games can still [[turn]] out a small , [[personal]] [[film]] with an emotional [[wallop]] . 

[[shows]] that the [[directors]] of such [[tinseltown]] blockbusters as patriot games can still [[turning]] out a small , [[individual]] [[movies]] with an emotional [[batting]] . 


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (90%)]] --> [[Negative (53%)]]

of [[saucy]] 

of [[smutty]] 


--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (99%)]] --> [[Positive (81%)]]

a [[depressed]] [[fifteen-year-old]] 's [[suicidal]] poetry 

a [[bummed]] [[fifteenyear-old]] 's [[kamikaze]] poetry 




[Succeeded / Failed / Skipped / Total] 8 / 1 / 1 / 10: 100%|██████████| 10/10 [00:01<00:00,  6.71it/s]

--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (79%)]] --> [[Negative (97%)]]

are more [[deeply]] thought through than in most ` right-thinking ' films 

are more [[severely]] thought through than in most ` right-thinking ' films 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 1      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 10.0%  |
| Attack success rate:          | 88.89% |
| Average perturbed word %:     | 31.93% |
| Average num. words per input: | 9.5    |
| Avg num queries:              | 29.44  |
+-------------------------------+--------+


##Recipe 12
            WordSwapQWERTY
            MaxModificationRate(0.1, 1),StopwordModification(),WordEmbeddingDistance(min_cos_sim=0.9)

In [54]:
class Recipe12(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
        #transformation =  WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        transformation = WordSwapQWERTY()

        constraints = [MaxModificationRate(0.1, 1),StopwordModification(), WordEmbeddingDistance(min_cos_sim=0.9)]

        goal_function = UntargetedClassification(model_wrapper)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

In [55]:
attack = Recipe12.build(model_wrapper)
attacker = Attacker(attack, dataset)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'allennlp.models.basic_classifier.BasicClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapQWERTY
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): MaxModificationRate(
        (max_rate):  0.1
        (min_threshold):  1
      )
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 2 / 0 / 2:  20%|██        | 2/10 [00:00<00:00, 14.21it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (95%)]] --> [[[FAILED]]]

hide new secretions from the parental units 


--------------------------------------------- Result 2 ---------------------------------------------
[[Negative (96%)]] --> [[[FAILED]]]

contains no wit , only labored gags 




[Succeeded / Failed / Skipped / Total] 0 / 3 / 1 / 4:  40%|████      | 4/10 [00:00<00:00, 16.82it/s]

...

[Succeeded / Failed / Skipped / Total] 0 / 5 / 1 / 6:  60%|██████    | 6/10 [00:00<00:00, 15.80it/s]

--------------------------------------------- Result 5 ---------------------------------------------
[[Negative (98%)]] --> [[[FAILED]]]

on the worst revenge-of-the-nerds clichés the filmmakers could dredge up 


--------------------------------------------- Result 6 ---------------------------------------------
[[Negative (99%)]] --> [[[FAILED]]]

that 's far too tragic to merit such superficial treatment 




[Succeeded / Failed / Skipped / Total] 0 / 9 / 1 / 10: 100%|██████████| 10/10 [00:00<00:00, 12.83it/s]

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (98%)]] --> [[[FAILED]]]

demonstrates that the director of such hollywood blockbusters as patriot games can still turn out a small , personal film with an emotional wallop . 


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (90%)]] --> [[[FAILED]]]

of saucy 


--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (99%)]] --> [[[FAILED]]]

a depressed fifteen-year-old 's suicidal poetry 


--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (79%)]] --> [[[FAILED]]]

are more deeply thought through than in most ` right-thinking ' films 



+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: 


/usr/local/lib/python3.7/dist-packages/textattack/metrics/attack_metrics/words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


##Recipe 13         
                    TargetedClassification

In [59]:
class Recipe13(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
        transformation = WordSwapWordNet()
        constraints = [RepeatModification(), StopwordModification()]
        goal_function = TargetedClassification(model_wrapper)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)


In [60]:
attack = Recipe13.build(model_wrapper)
attacker = Attacker(attack, dataset)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'allennlp.models.basic_classifier.BasicClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.targeted_classification.TargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  TargetedClassification(
    (target_class):  0
  )
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 0 / 2 / 2:  20%|██        | 2/10 [00:00<00:00, 104.61it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (95%)]] --> [[[SKIPPED]]]

hide new secretions from the parental units 


--------------------------------------------- Result 2 ---------------------------------------------
[[Negative (96%)]] --> [[[SKIPPED]]]

contains no wit , only labored gags 




[Succeeded / Failed / Skipped / Total] 1 / 0 / 5 / 6:  60%|██████    | 6/10 [00:00<00:00, 17.19it/s]

--------------------------------------------- Result 3 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (68%)]]

that [[loves]] its [[characters]] and [[communicates]] something [[rather]] beautiful about [[human]] nature 

that [[bed]] its [[type]] and [[pass]] something [[instead]] beautiful about [[man]] nature 


--------------------------------------------- Result 4 ---------------------------------------------
[[Positive (82%)]] --> [[[SKIPPED]]]

remains utterly satisfied to remain the same throughout 


--------------------------------------------- Result 5 ---------------------------------------------
[[Negative (98%)]] --> [[[SKIPPED]]]

on the worst revenge-of-the-nerds clichés the filmmakers could dredge up 


--------------------------------------------- Result 6 ---------------------------------------------
[[Negative (99%)]] --> [[[SKIPPED]]]

that 's far too tragic to merit such superficial treatment 




[Succeeded / Failed / Skipped / Total] 4 / 0 / 6 / 10: 100%|██████████| 10/10 [00:00<00:00, 11.37it/s]

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (98%)]] --> [[Negative (50%)]]

[[demonstrates]] that the [[director]] of such [[hollywood]] blockbusters as patriot games can still turn out a small , personal [[film]] with an emotional [[wallop]] . 

[[evidence]] that the [[manager]] of such [[Hollywood]] blockbusters as patriot games can still turn out a small , personal [[shoot]] with an emotional [[impact]] . 


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (90%)]] --> [[Negative (81%)]]

of [[saucy]] 

of [[impudent]] 


--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (99%)]] --> [[[SKIPPED]]]

a depressed fifteen-year-old 's suicidal poetry 


--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (79%)]] --> [[Negative (51%)]]

are 